# SQL Data Analysis

To start with, if you felt the class was unclear, check out the following tutorial: https://mode.com/sql-tutorial/introduction-to-sql/

Now! We'll be using sqlite to access a database. Start by downloading the sql lite file and putting it in the same directory as this notebook: https://www.kaggle.com/datasets/kaggle/sf-salaries (hit the 'download' button in the upper right). Check out the description of the data so you know the table / column names.

The following code will use sqlite to create a database connection.


In [54]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("database.sqlite")
crsr = conn.cursor()


Before we proceed, please note that every task **must be completed using a single SQL query**, unless mentioned otherwise or given as prompt.     
(e.g. using print statements are fine, but you should not be using **pandas library** to work with the dataset.). You can use Print statement with the single SQL question.

# Section 01: Exploration

Problem 1:


Try to create a query that gives you a data frame of the **EmployeeName, JobTitle, and BasePay** from the salaries table.Further limit the table to the year 2012, employees making under 150,000, and sort in descending order by salary.

In [55]:
query = '''
          SELECT EmployeeName, JobTitle, BasePay
          FROM salaries
          WHERE Year = 2012 AND BasePay < 150000
          ORDER BY BasePay DESC
        '''

df = pd.read_sql(query, conn)
df.head()

,EmployeeName,JobTitle,BasePay
0,Julie Kirschbaum,Project Manager 3,149881.06
1,Eric Jamison,Senior Physician Specialist,149746.72
2,Daisy Mae Quitoriano,Nurse Manager,149657.87
3,Ruth Armstrong,Nurse Practitioner,149169.48
4,Stuart Washington,Senior Physician Specialist,148811.73


# Section 02: Aggregation  



Problem 2:

Produce and print the head of a dataframe that shows the average pay for each year (only use a single, simple query). Your result should have a column for the year and a column for the average base pay.

In [56]:
query = '''
          SELECT Year, ROUND(AVG(BasePay), 2) as AvgBasePay
          FROM salaries
          GROUP BY Year
        '''

df = pd.read_sql(query, conn)
df.head()

,Year,AvgBasePay
0,2011,63595.96
1,2012,65436.41
2,2013,68509.83
3,2014,66557.44


**(4 POINTS)** Problem 3:

Create a dataframe with averages of base pay, averages of benefits, and averages of overtime for each job title, as well as a column with the sum of these three values.

In [57]:
query = '''
          SELECT  JobTitle, ROUND(AVG(BasePay), 2) as AvgBasePay,
            ROUND(AVG(Benefits), 2) as AvgBenefits,
            ROUND(AVG(OvertimePay), 2) as AvgOvertimePay,
            ROUND((AVG(BasePay) + AVG(Benefits) + AVG(OvertimePay)), 2) as Sum
          FROM salaries
          GROUP BY JobTitle
        '''
df = pd.read_sql(query, conn)
df.head()


,JobTitle,AvgBasePay,AvgBenefits,AvgOvertimePay,Sum
0,ACCOUNT CLERK,43300.81,0.00,373.20,43674.01
1,ACCOUNTANT,46643.17,0.00,0.00,46643.17
2,ACCOUNTANT INTERN,28732.66,0.00,24.43,28757.09
3,"ACPO,JuvP, Juv Prob (SFERS)",62290.78,17975.59,0.00,80266.37
4,ACUPUNCTURIST,66374.40,0.00,0.00,66374.40


 # Section 03: Table Creation

Problem 4: Create own Table

Now we'll create our own table in our database. Separate the Salaries table by Year, and add it back to the database. (You may use basic python to complete the task, however, should still use SQL to query the data.)


To do this, you'll have to learn how to create a table in SQL. You will likely need this function (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html), which should make the process simple.

Here's the basic structure of how you'll want to do this...

1. Through some form of loop, collect all the salaries in a given year.
2. Write that data to the sql connection using the function linked above. Make sure to give it a unique name!

In [58]:
for y in ['2011','2012','2013','2014']:
    query = '''
              SELECT *
              FROM salaries
              WHERE Year = {y}
            '''.format(y=y)
    df = pd.read_sql(query, conn)
    df.to_sql(name='Y'+y, con=conn, if_exists='replace')

**do not modify below code** (ONLY FOR DEBUGGING PURPOSE FOR PROBLEM 4)

*You can run the code below to check that things work. Your output should be similar to the expected output below... but the names of your new tables do not need to match exactly*.

In [59]:
### it is added to check if db has the tables added in above problem 8 ###
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

## if your above code is correct, in that case, running this cell will create output like following

##[('Salaries',), ('2011',), ('test',), ('Y2011',), ('Y2012',), ('Y2013',), ('Y2014',)]

[('Salaries',), ('Y2011',), ('Y2012',), ('Y2013',), ('Y2014',)]


# Section 04: Table Joining 

Problem 5:

We'll move on to a new dataset for the next steps.
Download the dataset from here (https://www.kaggle.com/datasets/luizpaulodeoliveira/imdb-project-sql) and load the sqlite file same as before. Start by just selecting everything in the "movies" table to see what it looks like.

In [60]:
conn = sqlite3.connect("movies.sqlite")
query = '''
          SELECT *
          FROM movies
        '''

df = pd.read_sql(query, conn)
df.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


Problem 6:

Create a dataframe that includes the entire contents of "movies" table as well as the director's name.

**Hint:** You may want to perform an inner join on the 'directors' table to complete this task. Investigate that table to learn more about this solution. Note that the movies table has a 'director_id' column which may help.

In [61]:
query = '''
        SELECT movies.*, directors.name as director_name
        FROM movies
        INNER JOIN directors ON movies.director_id = directors.id
        '''
df = pd.read_sql(query, conn)

# print(df.head(5))
# df.size
df.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id,director_name
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762,James Cameron
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763,Gore Verbinski
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764,Sam Mendes
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765,Christopher Nolan
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766,Andrew Stanton


# Section 05: Analysis  

The next few problems will be more involved! You'll need to combine some concepts you've learned. For each cell, show your work.
Remember, the answers should be in **a single query**.




Problem 7:

Which directors have the highest overall voting average? - show the top 5 directors' name and their average rating

In [62]:
query = '''
        SELECT directors.name as director_name,
          ROUND(AVG(movies.vote_average),2) as overall_vote_average
        FROM movies
        INNER JOIN directors ON movies.director_id = directors.id
        GROUP BY movies.director_id
        ORDER BY ROUND(AVG(movies.vote_average),2) DESC
        LIMIT 5
        '''
df = pd.read_sql(query, conn)

df.head(10)

,director_name,overall_vote_average
0,Gary Sinyor,10.00
1,Rohit Jugraj,9.50
2,Lance Hool,9.30
3,Floyd Mutrux,8.50
4,Tim McCanlies,8.45


Problem 08:

What are the top five directors by how much their average budget is?

In [63]:
query = '''
        SELECT directors.name as director_name,
          ROUND(AVG(budget), 2) as average_budget
        FROM movies
        INNER JOIN directors ON movies.director_id = directors.id
        GROUP BY director_id
        ORDER BY ROUND(AVG(budget), 2) DESC
        LIMIT 5
        '''
df = pd.read_sql(query, conn)

df.head(10)

,director_name,average_budget
0,Byron Howard,2.600000e+08
1,Lee Unkrich,2.000000e+08
2,Dan Scanlon,2.000000e+08
3,David Yates,1.933333e+08
4,Brenda Chapman,1.850000e+08
